In [1]:
import pandas as pd, numpy as np 
pd.set_option('display.max_columns', None)
import sqlalchemy, sqlite3 

In [2]:
season = "2016_17" 
dir = f"../../static/CSV/{season}/" 

names = ['pergame_'+season, 'advanced_'+season, 'per100_'+season, 'playbyplay_'+season, 'totals_'+season]  
dfs = {} 

invalid = ['-9999', 'Unnamed', 'Rk', 'Player-additional']

# Yuck I know triple nested for loop 
for name in names:
    dfs[name] = pd.read_csv(dir+name+'.csv') 
    for inv in invalid:
        for col in dfs[name].columns:
            if inv in col or col in invalid:
                
                dfs[name] = dfs[name].drop([col], axis=1)
                
dfs['totals_'+season] = dfs['totals_'+season].loc[:,['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'MP', 'FG', 'FGA', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB']] 
dfs['totals_'+season].columns = 'T' + dfs['totals_'+season].columns 
dfs['totals_'+season]['Player'] =  dfs['pergame_'+season]['Player']
dfs['totals_'+season] 

,TPTS,TTRB,TAST,TSTL,TBLK,TTOV,TPF,TMP,TFG,TFGA,T3PA,T2P,T2PA,TFT,TFTA,TORB,TDRB,Player
0,406,86,40,37,8,33,114,1055,134,341,247,40,94,44,49,18,68,Álex Abrines
1,222,115,18,14,15,21,67,558,70,170,90,33,80,45,60,20,95,Quincy Acy
2,905,613,86,89,78,146,195,2389,374,655,1,374,654,157,257,281,332,Steven Adams
3,515,125,78,21,6,42,104,1580,185,420,151,123,269,83,93,9,116,Arron Afflalo
4,207,177,12,20,22,31,77,584,89,178,4,89,174,29,40,46,131,Alexis Ajinça
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,639,405,99,62,58,65,189,1725,253,443,1,253,442,133,196,135,270,Cody Zeller
482,178,124,42,7,21,20,61,525,78,158,1,78,157,22,39,43,81,Tyler Zeller
483,23,35,4,2,5,3,17,108,10,31,0,10,31,3,5,11,24,Stephen Zimmerman
484,240,125,36,15,16,40,78,843,88,221,99,55,122,31,40,15,110,Paul Zipser


In [3]:
def combine_dfs(dfs, names):
    curr = None
    for name in names:
        if curr is None: 
            curr = dfs[name]
        else:
            diff = dfs[name].columns.difference(curr.columns).tolist()
            diff.append('Player') 
            # print(diff)
            curr = pd.merge(curr, dfs[name].loc[:,diff], on = 'Player') 
    return curr     

In [4]:
master = combine_dfs(dfs, names) 
master

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,3PAr,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,DRtg,ORtg,And1,BadPass,Blkd,C%,LostBall,Off.,Off..1,On-Off,OnCourt,PF%,PG%,PGA,SF%,SG%,Shoot,Shoot.1,T2P,T2PA,T3PA,TAST,TBLK,TDRB,TFG,TFGA,TFT,TFTA,TMP,TORB,TPF,TPTS,TSTL,TTOV,TTRB
0,Álex Abrines,SG,23,OKC,68,6,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,1.4,0.426,0.531,0.6,0.7,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,0.724,5.5,0.6,-1.6,-0.4,7.1,0.9,0.144,-1.3,1.9,1.2,10.1,1.7,8.3,4.5,0.560,15.9,0.1,2.1,0.096,110,113.0,4,17,8,NaN,7,4,10,-2.2,-1.2,NaN,NaN,94,24%,76%,69,15,40,94,247,40,8,68,134,341,44,49,1055,18,114,406,37,33,86
1,Quincy Acy,PF,26,TOT,38,1,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,0.529,4.9,2.0,-2.1,-0.6,18.0,0.5,0.353,-1.5,3.9,0.5,11.8,1.2,9.7,11.0,0.565,16.8,0.0,0.9,0.082,110,109.0,7,5,8,17%,7,6,5,2.8,-3.9,81%,NaN,42,2%,NaN,31,27,33,80,90,18,15,95,70,170,45,60,558,20,67,222,14,21,115
2,Steven Adams,C,23,OKC,80,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,8.2,0.572,0.571,2.0,3.2,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3,0.002,5.4,2.6,-0.2,0.0,15.4,3.1,0.392,-0.2,13.0,3.3,16.5,1.8,16.0,14.2,0.589,16.2,1.1,6.5,0.130,107,114.0,28,37,43,100%,57,32,7,7.7,3.5,NaN,NaN,217,NaN,NaN,75,118,374,654,1,86,78,332,374,655,157,257,2389,281,195,905,89,146,613
3,Arron Afflalo,SG,31,SAC,61,45,25.9,3.0,6.9,0.440,1.0,2.5,0.411,2.0,4.4,0.457,0.514,1.4,1.5,0.892,0.1,1.9,2.0,1.3,0.3,0.1,0.7,1.7,8.4,0.360,7.4,0.3,-3.6,-1.5,8.4,0.2,0.221,-2.1,0.7,1.2,8.9,0.7,8.4,4.6,0.559,14.4,-0.7,1.4,0.043,116,109.0,7,18,28,NaN,10,8,3,-5.8,-7.5,1%,1%,185,57%,41%,52,32,123,269,151,78,6,116,185,420,83,93,1580,9,104,515,21,42,125
4,Alexis Ajinça,C,28,NOP,39,15,15.0,2.3,4.6,0.500,0.0,0.1,0.000,2.3,4.5,0.511,0.500,0.7,1.0,0.725,1.2,3.4,4.5,0.3,0.5,0.6,0.8,2.0,5.3,0.022,3.1,3.1,-3.3,0.7,23.8,0.9,0.225,-4.0,8.3,0.0,12.9,1.7,13.7,16.0,0.529,17.2,-0.2,1.0,0.080,104,101.0,7,7,14,100%,8,8,0,-3.6,-5.0,NaN,NaN,27,NaN,NaN,45,22,89,174,4,12,22,131,89,178,29,40,584,46,77,207,20,31,177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,Cody Zeller,C,24,CHO,62,58,27.8,4.1,7.1,0.571,0.0,0.0,0.000,4.1,7.1,0.572,0.571,2.1,3.2,0.679,2.2,4.4,6.5,1.6,1.0,0.9,1.0,3.0,10.3,0.002,9.1,3.0,0.2,1.1,17.3,2.2,0.442,-0.9,8.6,3.4,16.7,1.8,10.9,12.9,0.604,15.5,1.0,5.6,0.157,107,121.0,21,19,56,73%,12,29,5,10.3,6.2,27%,NaN,238,NaN,NaN,98,85,253,442,1,99,58,270,253,443,133,196,1725,135,189,639,62,65,405
482,Tyler Zeller,C,27,BOS,51,5,10.3,1.5,3.1,0.494,0.0,0.0,0.000,1.5,3.1,0.497,0.494,0.4,0.8,0.564,0.8,1.6,2.4,0.8,0.1,0.4,0.4,1.2,3.5,0.006,12.2,3.3,-2.4,-0.2,17.0,0.6,0.247,-2.2,9.2,0.5,13.0,0.7,10.2,13.2,0.508,16.5,0.0,1.0,0.094,109,109.0,5,11,16,100%,3,6,3,-3.6,-0.1,NaN,NaN,99,NaN,NaN,36,16,78,157,1,42,21,81,78,158,22,39,525,43,61,178,7,20,124
483,Stephen Zimmerman,C,20,ORL,19,0,5.7,0.5,1.6,0.323,0.0,0.0,NaN,0.5,1.6,0.323,0.323,0.2,0.3,0.600,0.6,1.3,1.8,0.2,0.1,0.3,0.2,0.9,1.2,0.000,5.3,3.7,-8.3,-1.7,24.9,0.1,0.161,-6.6,10.8,-0.1,7.3,0.9,8.3,17.6,0.346,14.8,-0.2,0.0,-0.005,108,86.0,1,1,7,100%,0,1,0,-11.0,-17.7,NaN,NaN,9,NaN,NaN,12,2,10,31,0,4,5,24,10,31,3,5,108,11,17,23,2,3,35
484,Paul Zipser,SF,22,CHI,44,18,19.2,2.0,5.0,0.398,0.8,2.3,0.333,1.3,2.8,0.451,0.473,0.7,0.9,0.775,0.3,2.5,2.8,0.8,0.3,0.4,0.9,1.8,5.5,0.448,6.1,1.5,-4.4,-0.2,14.2,0.8,0.181,-4.2,1.9,-0.3,6.9,0.9,14.4,8.0,0.503,14.4,-0.5,0.5,0.030,109,96.0,4,19,10,NaN,11,2,2,1.5,1.3,15%,NaN,85,85%,NaN,46,17,55,122,99,36,16,110,88,221,31,40,843,15,78,240,15,40,125


In [5]:
engine = sqlalchemy.create_engine('sqlite:///../../DB/ballbase.db') 
for name in names:
    dfs[name].to_sql(name, con = engine, if_exists = 'replace', index = False)  

In [6]:
master.to_sql('master_'+season, con=engine, if_exists='replace', index=False) 
'master_'+season

'master_2016_17'